In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import numpy as np
import os
import re

# Mapping from amino acid characters to integers
aa_map = {
    'A': 0, 'C': 1, 'D': 2, 'E': 3, 'F': 4,
    'G': 5, 'H': 6, 'I': 7, 'K': 8, 'L': 9,
    'M': 10, 'N': 11, 'P': 12, 'Q': 13, 'R': 14,
    'S': 15, 'T': 16, 'V': 17, 'W': 18, 'Y': 19,
    'X': 20,  'B': 21,  'Z': 22,  'J': 23,  '-': 24,
}

# Mapping for secondary structure labels
ss_map = {'H': 0, 'E': 1, 'C': 2}

# Custom dataset class for handling protein data
class ProteinDataset(Dataset):
    def __init__(self, seqs_csv_file, train_directory, labels_csv_file=None, normalization_method='min-max'):
        # Reading the sequence feature file
        self.seqs_data = pd.read_csv(seqs_csv_file)
        self.protein_data = {}
        
        # Getting residue number, amino acid and PSSM profiles for every protein
        for filename in os.listdir(train_directory):
            if filename.endswith(".csv"):
                # Given the folder name, checks all folders that terminate with either _train or _test
                pdb_id = re.split(r'_train|_test', filename)[0] 
                self.protein_data[pdb_id] = pd.read_csv(os.path.join(train_directory, filename))
                
        # Getting the ground labels (Secondary Structure)
        if labels_csv_file:
            self.labels_data = pd.read_csv(labels_csv_file)
        else:
            self.labels_data = None
        self.aa_mapping = aa_map
        
        # Setting the normalization method (default -> min-max)
        self.normalization_method = normalization_method

    def get_seq_encoding(self, sequence):
        # Function to encode the sequence based on the amino acid mapping
        seq_len = len(sequence)
        aa_map_len = len(self.aa_mapping)
        encoded_seq = np.zeros((seq_len, aa_map_len), dtype=int)
        
        # Maps the corresponding value of Amino Acid - X for unknown amino acids
        for i, aa in enumerate(sequence):
            index = self.aa_mapping.get(aa, self.aa_mapping['X'])
            encoded_seq[i, index] = 1
        return encoded_seq

    def get_norm_pssm(self, pssm):
        # Function to normalize the PSSM Profile
        numeric_columns = pssm[:, 2:]
        pssm_num = numeric_columns.astype(np.float32)

        if self.normalization_method == 'min-max':
            # Normalizing the PSSM profile using min-max algorithm
            pssm_min = pssm_num.min(axis=0)
            pssm_max = pssm_num.max(axis=0)
            pssm_range = np.where(pssm_max - pssm_min == 0, 1, 
                                  pssm_max - pssm_min)
            normalized_pssm = (pssm_num - pssm_min) / pssm_range
        elif self.normalization_method == 'z-score':
            # Normalizing the PSSM profile using z-score algorithm
            pssm_mean = pssm_numeric.mean(axis=0)
            pssm_std = pssm_numeric.std(axis=0)
            pssm_std = np.where(pssm_std == 0, 1, pssm_std)
            normalized_pssm = (pssm_numeric - pssm_mean) / pssm_std
        else:
            normalized_pssm = pssm_numeric

        return normalized_pssm

    def __len__(self):
        return len(self.seqs_data)

    def __getitem__(self, idx):
        # Getting the PDB ID, Sequence 
        pdb_id = self.seqs_data.iloc[idx]['PDB_ID']
        seqs_data = self.seqs_data.iloc[idx]['SEQUENCE']
        
        # Encoding the sequence
        encoded_seq = self.get_seq_encoding(seqs_data)
        pssm = self.protein_data[pdb_id].values
        norm_pssm = self.get_norm_pssm(pssm)
                               
        encoded_seq_tensor = torch.tensor(encoded_seq, dtype=torch.float32)
        norm_pssm_tensor = torch.tensor(norm_pssm, dtype=torch.float32)

        # Getting the ground truth labels (secondary structure)
        if self.labels_data is not None:
            label_seq = self.labels_data.iloc[idx]['SEC_STRUCT']
            
            # Encoding the labels using secondarys structure map 
            label_num = [ss_map[char] for char in label_seq]
            
            # Converting the encoded labels to pytorch tensors
            label_tensor = torch.tensor(label_num, dtype=torch.long)
                    
            return (pdb_id, encoded_seq_tensor, norm_pssm_tensor, label_tensor)

        return (pdb_id, encoded_seq_tensor, norm_pssm_tensor)
    
def custom_collate_fn_nolabel(batch):
    # Collate function for DataLoader that doesn't have labels
    ids, sequences, pssms = zip(*batch)
    
    # Padding the sequences to match the max sequence length
    pad_seq_list = [seq.clone().detach() for seq in sequences]
    sequences_padded = pad_sequence(pad_seq_list, batch_first=True)
    
    # Padding the pssms
    pssms_padded = torch.tensor(pssms)
    return ids, sequences_padded, pssms_padded

def custom_collate_fn(batch):
    # Collate function for DataLoader with labels
    _, sequences, pssms, labels_list = zip(*batch)
    
    # Padding the sequences to match the max sequence length
    pad_seq_list = [seq.clone().detach() for seq in sequences]
    sequences_padded = pad_sequence(pad_seq_list, batch_first=True)
    
    # Padding the pssms
    pd_pssms_list = [pssm.clone().detach() for pssm in pssms]
    pssms_padded = pad_sequence(pd_pssms_list, batch_first=True)
    
    # Padding the secondary structures to match the max length
    if labels_list[0] is not None:
        pad_labels_lst = [label.clone().detach() for label in labels_list]
        labels_padded = pad_sequence(pad_labels_lst, batch_first=True)
    else:
        labels_padded = None
        
    # Adding a mask for the padded labels
    mask = [torch.ones(len(label), dtype=torch.uint8) for label in labels_list]
    mask_padded = pad_sequence(mask, batch_first=True, padding_value=0)
    
    return sequences_padded, pssms_padded, labels_padded, mask_padded


class ProteinModel(nn.Module):
    # Class to construct a Fully Convolutional Protein Model using PyTorch
    def __init__(self, num_classes=3, input_channels=20):
        super(ProteinModel, self).__init__()
        
        # Creating 3 1D CNNs
        self.conv1 = nn.Conv1d(in_channels=input_channels, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.final_conv = nn.Conv1d(in_channels=256, out_channels=num_classes, kernel_size=1)

    def forward(self, x):
        # Forward method for the CNN network
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        
        # Final conv layer
        x = self.final_conv(x)
        x = x.transpose(1, 2)
        return x
    
# Function to train the model
def model_train(model, criterion, optimizer, train_loader, num_epochs=10):
    # If Nvidia CUDA is available, use that otherwise use CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Moving the model to GPU
    model.to(device)
    
    # Train the network over multiple epochs
    for epoch in range(num_epochs):
        
        # Training the model
        model.train()
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0

        for sequences, pssms, labels, _ in train_loader:
            
            # Getting the input pssms
            inputs = pssms.permute(0, 2, 1).to(device)

            # Initializing optimizer
            optimizer.zero_grad()

            # Model outputs
            outputs = model(inputs)
            
            # Moving the labels to GPU
            labels = labels.to(device)
            
            # Calculating the loss
            loss = criterion(outputs.transpose(1, 2), labels)

            # Back Propagation to update the weights
            loss.backward()
            optimizer.step()

            # Calculating the cumulative loss
            running_loss += loss.item() * inputs.size(0)

            # Getting the predictions
            _, predicted = torch.max(outputs, 2)
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.numel()

        # Calculating the loss per epoch
        td = train_loader.dataset
        epoch_loss = running_loss / len(td)
        epoch_acc = correct_preds / total_preds
        
        print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_acc:.4f}')
        
# Main part of the script
seqs_csv_dir = '/kaggle/input/deep-learning-for-msc-202324/seqs_train.csv'
labels_csv_dir = '/kaggle/input/deep-learning-for-msc-202324/labels_train.csv'
train_path = '/kaggle/input/deep-learning-for-msc-202324/train'
test_path = '/kaggle/input/deep-learning-for-msc-202324/test'

# Load data
dataset = ProteinDataset(seqs_csv_file=seqs_csv_dir, train_directory=train_path, labels_csv_file=labels_csv_dir)
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

# Training DataLoader
train_loader = DataLoader(train_data, batch_size=4, shuffle=True, collate_fn=custom_collate_fn)

# Validation DataLoader
val_loader = DataLoader(test_data, batch_size=4, shuffle=False, collate_fn=custom_collate_fn)

# Training the model
model = ProteinModel()

criterion = nn.CrossEntropyLoss()

# According to Ax, these are best hyperparameters, however, these don't give good accuracy
# Best Parameters: {'lr': 0.00010943465377382328, 'weight_decay': 0.0, 'num_epochs': 9}
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001, weight_decay=0.0)

# Best num epochs
num_epochs = 10

model_train(model, criterion, optimizer, train_loader, num_epochs)

def calculate_q3_accuracy(predictions, ground_truth):
    # Function to calculate Q3 Accuracy
    correct = 0
    total = 0
    for pred, truth in zip(predictions, ground_truth):
        for p, t in zip(pred, truth):
            if p == t:
                correct += 1
            total += 1
    q3_accuracy = correct / total
    return q3_accuracy

# Test model and calculate Q3 accuracy on Validation Set
def test_model_and_calc_q3(model, val_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    predictions = []
    ground_truth = []

    with torch.no_grad():
        for sequences, pssms, labels, _ in val_loader:
            inputs = pssms.permute(0, 2, 1).to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 2)

            predicted = predicted.view(-1)

            for pred, label in zip(predicted, labels.view(-1)):
                pred_index = pred.item()
                structure_label = ['H', 'E', 'C'][pred_index]
                predictions.append(structure_label)

                gt_label = ['H', 'E', 'C'][label.item()]
                ground_truth.append(gt_label)

    q3_accuracy = calculate_q3_accuracy(predictions, ground_truth)
    print(f'Q3 Accuracy on Validation Set: {q3_accuracy:.4f}')
    return q3_accuracy

q3_accuracy = test_model_and_calc_q3(model, val_loader)

def test(model, test_dataset, output_file='/kaggle/working/submission.csv'):
    # If Nvidia CUDA is available, use that otherwise use CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Evaluate the model (no training - so backprop won't be performed)
    model.eval()
    predictions = []

    with torch.no_grad():
        # Iterating through the test dataset
        for i in range(len(test_dataset)):
            
            # Getting the PDB ID and PSSM values
            pdb_id, _, pssm = test_dataset[i]
            input_pssm = pssm.unsqueeze(0).permute(0, 2, 1).to(device)

            # Getting outputs from the model
            outputs = model(input_pssm)
            _, predicted = torch.max(outputs, 2)

            seq_len = pssm.shape[0]
            # Iterating through the sequences
            for j in range(seq_len):
                # Forming predictions with the format PDB_ID_RESIDUE_NUMBER, SECONDARY_STRUCTURE
                residue_id = f"{pdb_id}_{j+1}"
                struct_label = ['H', 'E', 'C'][predicted[0, j].item()]
                predictions.append([residue_id, struct_label])

    # Saving the predictions to submission.csv
    submission_df = pd.DataFrame(predictions, columns=['ID', 'STRUCTURE'])
    submission_df.to_csv(output_file, index=False)
    print(f'Submission file saved to {output_file}')

seqs_test_dir = '/kaggle/input/deep-learning-for-msc-202324/seqs_test.csv'
test_path = '/kaggle/input/deep-learning-for-msc-202324/test'

test_dataset = ProteinDataset(seqs_csv_file=seqs_test_dir, train_directory=test_path)

test(model, test_dataset)

Epoch 1/10, Train Loss: 0.4099, Train Accuracy: 0.8374
Epoch 2/10, Train Loss: 0.3587, Train Accuracy: 0.8587
Epoch 3/10, Train Loss: 0.3456, Train Accuracy: 0.8640
Epoch 4/10, Train Loss: 0.3361, Train Accuracy: 0.8687
Epoch 5/10, Train Loss: 0.3317, Train Accuracy: 0.8699
Epoch 6/10, Train Loss: 0.3287, Train Accuracy: 0.8715
Epoch 7/10, Train Loss: 0.3245, Train Accuracy: 0.8731
Epoch 8/10, Train Loss: 0.3194, Train Accuracy: 0.8750
Epoch 9/10, Train Loss: 0.3160, Train Accuracy: 0.8764
Epoch 10/10, Train Loss: 0.3162, Train Accuracy: 0.8764
Q3 Accuracy on Validation Set: 0.8705
Submission file saved to /kaggle/working/submission.csv


### Hyperparameter Tuning

In [ ]:
!pip install ax-platform

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import numpy as np
import os
import re
from ax import optimize

def train_evaluate(parameterization):
    print("Parameterization:", parameterization)
    lr = parameterization.get('lr', 0.001)
    weight_decay = parameterization.get('weight_decay', 0.0)
    num_epochs = parameterization.get('num_epochs', 10)

    model = ProteinModel()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay)

    train_model(model, criterion, optimizer, train_loader, num_epochs)

    q3_accuracy = test_model_and_calc_q3(model, val_loader)  # Calculate Q3 accuracy
    print("q3_accuracy:", q3_accuracy)

    return {"q3_accuracy": (q3_accuracy, 0.0)}  # Returning as a dictionary

search_space = [
    {"name": "lr", "type": "range", "bounds": [1e-5, 1e-1], "log_scale": True},
    {"name": "weight_decay", "type": "range", "bounds": [0.0, 0.1]},
    {"name": "num_epochs", "type": "range", "bounds": [9, 10]},
]

# seqs_csv_dir = '/kaggle/input/deep-learning-for-msc-202324/seqs_train.csv'
# labels_csv_dir = '/kaggle/input/deep-learning-for-msc-202324/labels_train.csv'
# train_path = '/kaggle/input/deep-learning-for-msc-202324/train'
# test_path = '/kaggle/input/deep-learning-for-msc-202324/test'

seqs_csv_dir = '/Users/dhanyasasikumar/Desktop/Computing Science PGT Course/Semester 2/Deep Learning/Coursework/dataset/deep-learning-for-msc-202324/seqs_train.csv'
labels_csv_dir = '/Users/dhanyasasikumar/Desktop/Computing Science PGT Course/Semester 2/Deep Learning/Coursework/dataset/deep-learning-for-msc-202324/labels_train.csv'
train_path = '/Users/dhanyasasikumar/Desktop/Computing Science PGT Course/Semester 2/Deep Learning/Coursework/dataset/deep-learning-for-msc-202324/train'
test_path = '/Users/dhanyasasikumar/Desktop/Computing Science PGT Course/Semester 2/Deep Learning/Coursework/dataset/deep-learning-for-msc-202324/test'


dataset = ProteinDataset(seqs_csv_file=seqs_csv_dir, train_directory=train_path, labels_csv_file=labels_csv_dir)
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

train_loader = DataLoader(train_data, batch_size=4, shuffle=True, collate_fn=custom_collate_fn)
val_loader = DataLoader(test_data, batch_size=4, shuffle=False, collate_fn=custom_collate_fn)

best_parameters, values, experiment, model = optimize(
    parameters=search_space,
    evaluation_function=train_evaluate,
    objective_name='q3_accuracy',
    total_trials=10,
)

print("Best Parameters:", best_parameters)
print("Objective Values:", values)

[INFO 03-18 14:25:13] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter lr. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-18 14:25:13] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter weight_decay. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-18 14:25:13] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter num_epochs. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-18 14:25:13] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='lr', parameter_type=FLOAT, range=[1e-05, 0.1], log_scale=True), RangeParameter(name='weight_decay', parameter_type=FLOAT, 

Parameterization: {'lr': 0.08265406608089564, 'weight_decay': 0.09397713541984559, 'num_epochs': 9}
Epoch 1/9, Loss: 251.5962, Accuracy: 0.6389
Epoch 2/9, Loss: 3.3524, Accuracy: 0.6308
Epoch 3/9, Loss: 20.4914, Accuracy: 0.6362
Epoch 4/9, Loss: 0.9169, Accuracy: 0.6422
Epoch 5/9, Loss: 0.9168, Accuracy: 0.6423
Epoch 6/9, Loss: 0.9128, Accuracy: 0.6466
Epoch 7/9, Loss: 0.9144, Accuracy: 0.6452
Epoch 8/9, Loss: 0.9114, Accuracy: 0.6462
Epoch 9/9, Loss: 0.9141, Accuracy: 0.6452


[INFO 03-18 14:26:39] ax.service.managed_loop: Running optimization trial 2...


Q3 Accuracy on Validation Set: 0.6388
q3_accuracy: 0.6387708592178695
Parameterization: {'lr': 0.0003738899150858358, 'weight_decay': 0.07317882720381022, 'num_epochs': 10}
Epoch 1/10, Loss: 0.6596, Accuracy: 0.6991
Epoch 2/10, Loss: 0.6303, Accuracy: 0.7273
Epoch 3/10, Loss: 0.6248, Accuracy: 0.7291
Epoch 4/10, Loss: 0.6282, Accuracy: 0.7283
Epoch 5/10, Loss: 0.6259, Accuracy: 0.7285
Epoch 6/10, Loss: 0.6220, Accuracy: 0.7310
Epoch 7/10, Loss: 0.6263, Accuracy: 0.7287
Epoch 8/10, Loss: 0.6253, Accuracy: 0.7292
Epoch 9/10, Loss: 0.6273, Accuracy: 0.7296
Epoch 10/10, Loss: 0.6217, Accuracy: 0.7318


[INFO 03-18 14:28:02] ax.service.managed_loop: Running optimization trial 3...


...

Q3 Accuracy on Validation Set: 0.7873
q3_accuracy: 0.7872668675649888
Parameterization: {'lr': 0.00035406362860655825, 'weight_decay': 0.004160757129606094, 'num_epochs': 10}
Epoch 1/10, Loss: 0.4631, Accuracy: 0.8134
Epoch 2/10, Loss: 0.4157, Accuracy: 0.8343
Epoch 3/10, Loss: 0.4177, Accuracy: 0.8343
Epoch 4/10, Loss: 0.4120, Accuracy: 0.8370
Epoch 5/10, Loss: 0.4147, Accuracy: 0.8365
Epoch 6/10, Loss: 0.4083, Accuracy: 0.8385
Epoch 7/10, Loss: 0.4108, Accuracy: 0.8378
Epoch 8/10, Loss: 0.4057, Accuracy: 0.8390
Epoch 9/10, Loss: 0.4096, Accuracy: 0.8388
Epoch 10/10, Loss: 0.4077, Accuracy: 0.8389


[INFO 03-18 14:36:52] ax.service.managed_loop: Running optimization trial 9...


Q3 Accuracy on Validation Set: 0.8358
q3_accuracy: 0.8357501767584895
Parameterization: {'lr': 0.00024200160205771219, 'weight_decay': 0.006901753888666292, 'num_epochs': 10}
Epoch 1/10, Loss: 0.4862, Accuracy: 0.8004
Epoch 2/10, Loss: 0.4382, Accuracy: 0.8260
Epoch 3/10, Loss: 0.4287, Accuracy: 0.8298
Epoch 4/10, Loss: 0.4240, Accuracy: 0.8324
Epoch 5/10, Loss: 0.4248, Accuracy: 0.8324
Epoch 6/10, Loss: 0.4186, Accuracy: 0.8341
Epoch 7/10, Loss: 0.4226, Accuracy: 0.8324
Epoch 8/10, Loss: 0.4273, Accuracy: 0.8315
Epoch 9/10, Loss: 0.4184, Accuracy: 0.8338
Epoch 10/10, Loss: 0.4194, Accuracy: 0.8344


[INFO 03-18 14:38:33] ax.service.managed_loop: Running optimization trial 10...


Q3 Accuracy on Validation Set: 0.8331
q3_accuracy: 0.8330885028041104
Parameterization: {'lr': 0.001492506197790241, 'weight_decay': 0.005777916503364055, 'num_epochs': 10}
Epoch 1/10, Loss: 0.4533, Accuracy: 0.8188
Epoch 2/10, Loss: 0.4307, Accuracy: 0.8289
Epoch 3/10, Loss: 0.4274, Accuracy: 0.8310
Epoch 4/10, Loss: 0.4260, Accuracy: 0.8315
Epoch 5/10, Loss: 0.4232, Accuracy: 0.8318
Epoch 6/10, Loss: 0.4212, Accuracy: 0.8334
Epoch 7/10, Loss: 0.4221, Accuracy: 0.8323
Epoch 8/10, Loss: 0.4217, Accuracy: 0.8327
Epoch 9/10, Loss: 0.4231, Accuracy: 0.8327
Epoch 10/10, Loss: 0.4187, Accuracy: 0.8338
Q3 Accuracy on Validation Set: 0.8339
q3_accuracy: 0.8339036511782755
Best Parameters: {'lr': 0.00035406362860655825, 'weight_decay': 0.004160757129606094, 'num_epochs': 10}
Objective Values: ({'q3_accuracy': 0.8357496082954966}, {'q3_accuracy': {'q3_accuracy': 4.622222494204315e-09}})
